In [1]:
#!pip install imblearn
#!pip install sklearn

Importamos librerias

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report


/opt/conda/envs/geopandas/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Cargamos datos

In [3]:
original = pd.read_csv("adult_clean.csv") #Cargamos el original limpio

original["target"] = original["income_num"] #Hacemos de income el target
original.drop(["income_num","age",'hours_per_week','american_num'],axis=1,inplace=True) #Eliminamos los elementos innecesarios


#Cargamos el segundo
data = pd.read_csv("data.csv")
data = data[['age_cuartiles', 'cat_trabajo', 'US']]

In [4]:
for i in data.columns: #transferimos los datos al original
    original[i] = data[i]
    
data = original #Cambiamos el nombre del mismo

In [5]:
data.age_cuartiles = data.age_cuartiles.astype(str)
data.cat_trabajo = data.cat_trabajo.astype(str)

In [6]:
data

,workclass_num,education_num,marital_status_num,occupation_num,relationship_num,sex_num,capital_loss,capital_gain,target,age_cuartiles,cat_trabajo,US
0,0,9,0,0,0,1,4356,0,0,"(47.0, 90.0]",Menos 40,1
1,0,4,1,1,1,1,3900,0,0,"(47.0, 90.0]",Igual a 40,1
2,0,10,2,2,2,1,3900,0,0,"(37.0, 47.0]",Igual a 40,1
3,0,9,1,3,1,1,3770,0,0,"(28.0, 37.0]",Mayor a 40,1
4,0,6,2,4,1,0,3770,0,0,"(37.0, 47.0]",Igual a 40,1
5,1,16,3,2,3,1,3683,0,1,"(47.0, 90.0]",Menos 40,1
6,2,9,1,2,0,1,3683,0,0,"(47.0, 90.0]",Igual a 40,1
7,0,16,1,2,1,1,3004,0,1,"(37.0, 47.0]",Menos 40,1
8,3,15,3,2,0,0,2824,0,1,"(37.0, 47.0]",Mayor a 40,1
9,0,13,0,3,0,1,2824,0,1,"(47.0, 90.0]",Menos 40,1


Verificamos la desviacion de los datos.

In [7]:
data.target.value_counts(normalize=False)

0    22654
1     7508
Name: target, dtype: int64

In [8]:
data = pd.get_dummies(data)

Hacemos el split.

In [9]:
training_features, test_features, \
training_target, test_target, = train_test_split(data.drop(['target'], axis=1),
                                               data['target'],
                                               test_size = .2,
                                               random_state=12)


Instanciamos y fiteamos el smote a los datos train.

In [10]:
#Hacemos smote sobre nuestro train
sm = SMOTE(random_state=12,n_jobs=-1)
x_train_res, y_train_res = sm.fit_sample(training_features, training_target)

#Ordenamos nuestro test
test = pd.DataFrame(test_features)
test["target"] = test_target

#lo ordenamos en un solo lugar
adult_smote = pd.DataFrame(x_train_res)
adult_smote.columns = training_features.columns
adult_smote["target"] = y_train_res

Verificamos que los datos estan nivelados

In [11]:
adult_smote.target.value_counts()

1    18079
0    18079
Name: target, dtype: int64

Exportamos los datos a csv.

In [12]:
test.to_csv("test.csv", encoding='utf-8', index=False)
adult_smote.to_csv("data_smote.csv", encoding='utf-8', index=False)